In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
column_info = pd.read_csv('plan_summary.csv')
plan_data = pd.read_csv('../data_subsidy/Plan_Attributes_PUF_2016.csv')



/home/erichschulman/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (31,79,80,81,85,86,87,88,89,90,135,136,137,138) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
good_columns = column_info['Column_Name'][ (column_info['Missing_Values'] <= 3000) & (column_info['Missing_Values'] >= 0) & (column_info['Unique_Values'] >= 90) 
                  & (column_info['Unique_Values'] <= 370) ]
good_columns = list(good_columns)

#fix the set of columns to remove bad ones
bad_columns = ['SBCHavingSimplefractureCoinsurance', 
 'SBCHavingSimplefractureCopayment', 'SBCHavingSimplefractureDeductible', 'ServiceAreaID']
good_columns = list(set(good_columns) - set(bad_columns))


# https://stackoverflow.com/questions/4703390/how-to-extract-a-floating-number-from-a-string
clean_columns = plan_data[['ServiceAreaId','IssuerId']].copy()

for column in good_columns:
    #try to extract the text
    test_column = plan_data[column].copy()
    test_column = test_column.fillna(0)
    
    #if its just an int go with that
    int_column = test_column.apply(lambda x : float( (re.findall(r'^-?\d+(?:\.\d+)$',str(x)) +['0']) [0]) )
    if int_column.nunique() >= 3:
        clean_columns[column] = int_column
        
    #otherwise try something else
    dollar_column = test_column.apply(lambda x : int((re.findall('\\$(\\d+)',str(x) ) +['0'])[0]) )
    if dollar_column.nunique() >= 3:
        clean_columns[column] = dollar_column

print(clean_columns.columns)

Index(['ServiceAreaId', 'IssuerId', 'TEHBOutOfNetFamilyPerGroupMOOP',
       'TEHBInnTier1IndividualMOOP', 'TEHBOutOfNetIndividualMOOP',
       'TEHBOutOfNetFamilyPerPersonMOOP', 'SBCHavingaBabyCoinsurance',
       'SBCHavingaBabyCopayment', 'TEHBInnTier1FamilyPerPersonMOOP',
       'SBCHavingDiabetesCopayment', 'SBCHavingDiabetesDeductible',
       'TEHBCombInnOonFamilyPerPersonMOOP', 'SBCHavingaBabyDeductible',
       'EHBPercentTotalPremium', 'SBCHavingDiabetesCoinsurance',
       'TEHBCombInnOonFamilyPerGroupMOOP', 'TEHBCombInnOonIndividualMOOP',
       'TEHBInnTier1FamilyPerGroupMOOP',
       'OutOfServiceAreaCoverageDescription'],
      dtype='object')


In [4]:
clean_columns.describe()
clean_columns.to_csv('issuer_numeric.csv')